Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы 3*.Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import numpy as np
from pymongo import MongoClient
import pprint

In [2]:
def req_hh(lang, page):
    if lang == 'C#':
        lang = 'C%23'
    URL = f'https://m.hh.ru/vacancies?text={lang}&only_with_salary=true&area=1&page={page}'
    get = requests.get(URL)
    get.encoding = 'utf-8'
    html = get.text
    parsed_html = bs(html,'lxml')
    return parsed_html


def vacancy_search(money, collection):
    return collection.find( { '$or': [ {'min_salary': {'$gte': money} }, {'max_salary': {'$gte': money} } ] } )



Инициализация БД:

In [3]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['lesson_4']
lesson = db.jobs_list

In [4]:
REQUEST = 'Python'
NUMB_PAGES = 3

Парсинг HTML страницы и получение списка вакансий:

In [5]:
jobs_list = []
for page in range(NUMB_PAGES):
    
    parsed_html = req_hh(REQUEST, page)
    divs_with_job = parsed_html.find_all('ul')[0].find_all('li')
    
    for i in divs_with_job:
        job = {}
        spam_currency = i.find_all('span')
        min_salary = max_salary = np.nan
        if len(spam_currency) > 1:
            
            salary = list(i.find_all('span')[0].strings)[0]
            _str = salary.replace(u'\u2009','').replace(u'\n                    ','')

            if _str[:2] == 'от':
                min_salary = int(re.findall('\d+',_str)[0]) 
            elif _str[:2] == 'до':
                max_salary = int(re.findall('\d+',_str)[0])
            else:
                min_salary = int(re.findall('\d+',_str)[0])
                max_salary = int(re.findall('\d+',_str)[1])
            
            job['min_salary'] = min_salary
            job['max_salary'] = max_salary
            job['currency'] = i.find_all('span')[1].string
            job['description'] = i.find_all('div')[1].string[13:-9]
            job['url'] = i.find_all('a')[-1]['href']
            
            jobs_list.append(job)
            

Запись вакансий с проверкой на существование в базе (без индексов(хорошо бы с ними)):

In [6]:
for i in range(len(jobs_list)):
    if lesson.count_documents({'url': jobs_list[i]['url']}) == 0:
        lesson.insert_one(jobs_list[i])

Поиск вакансий с зарплатой выше заданной:

In [7]:
MONEY = 190000

In [8]:
for i in vacancy_search(MONEY, lesson):
    print(f'Вакансия: "{i["description"]}"\nЗарплата: min={i["min_salary"]}\tmax={i["max_salary"]}\nСсылка: {i["url"]}\n')

Вакансия: "Python Developer"
Зарплата: min=160000	max=250000
Ссылка: https://hh.ru/vacancy/32072763

Вакансия: "Разработчик Python"
Зарплата: min=160000	max=200000
Ссылка: https://hh.ru/vacancy/31624245

Вакансия: "Ведущий Python разработчик"
Зарплата: min=150000	max=250000
Ссылка: https://hh.ru/vacancy/32061372

Вакансия: "Разработчик Python/Django + React.js"
Зарплата: min=145000	max=255000
Ссылка: https://hh.ru/vacancy/31844761

Вакансия: "Разработчик Python"
Зарплата: min=150000	max=200000
Ссылка: https://hh.ru/vacancy/31566378

Вакансия: "Разработчик Python/Team-lead"
Зарплата: min=120000	max=200000
Ссылка: https://hh.ru/vacancy/31974215

Вакансия: "Full-stack разработчик (Python, JavaScript)"
Зарплата: min=100000	max=200000
Ссылка: https://hh.ru/vacancy/29195763

Вакансия: "Senior Python Developer"
Зарплата: min=150000	max=220000
Ссылка: https://hh.ru/vacancy/31987135

Вакансия: "Data Engineering Team Lead (Python / C++)"
Зарплата: min=300000	max=400000
Ссылка: https://hh.ru/vaca